In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import logging
import sys

project_root = Path("/Users/gabrielduarte/Documents/GitHub/SaezAtienzar/perinuclear_analysis")
sys.path.insert(0, str(project_root))
  
from perinuclear_analysis.core import BackgroundSubtractionConfig, PreprocessingConfig
from perinuclear_analysis.imaging_preprocessing.background_subtraction import BackgroundSubtractor
from perinuclear_analysis.data_processing import ImageLoader

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


/Users/gabrielduarte/Documents/GitHub/SaezAtienzar/perinuclear_analysis/perinuclear_analysis/__init__.py:41: UserWarning: MIPPreprocessor not available yet
  warnings.warn("MIPPreprocessor not available yet")
/Users/gabrielduarte/Documents/GitHub/SaezAtienzar/perinuclear_analysis/perinuclear_analysis/__init__.py:61: UserWarning: Phase 4 dependencies not installed: No module named 'perinuclear_analysis.ring_analysis'
Install with: pip install .[phase4]
  warnings.warn(
/Users/gabrielduarte/Documents/GitHub/SaezAtienzar/perinuclear_analysis/perinuclear_analysis/__init__.py:74: UserWarning: Phase 5 dependencies not installed: No module named 'perinuclear_analysis.signal_quantification'
Install with: pip install .[phase5]
  warnings.warn(


In [2]:
image_path = Path("/Users/gabrielduarte/Documents/GitHub/SaezAtienzar/perinuclear_analysis/data/ALIX/nd2/anti_ALIX_15_min_1.ome.tiff")

image_loader = ImageLoader()
loaded_data = image_loader.load_image(image_path)

INFO:perinuclear_analysis.data_processing.image_loader:Loaded metadata from converted TIFF: pixel_size=0.108333333333333μm
INFO:perinuclear_analysis.data_processing.image_loader:Successfully loaded TIFF image with shape: (23, 1800, 1800, 4)


In [ ]:
len(loaded_data.shape)

# channel_names = image_loader.get_channel_names()

image_loader.rename_channels(["LAMP1", "Phalloidin", "ALIX", "DAPI"])





INFO:perinuclear_analysis.data_processing.image_loader:Renamed channels to: ['LAMP1', 'Phalloidin', 'ALIX', 'DAPI']


In [6]:
channel_names = image_loader.get_channel_names()

channel_names

['LAMP1', 'Phalloidin', 'ALIX', 'DAPI']

In [9]:
bg_subtractor = BackgroundSubtractor()

pixel_size = image_loader.get_pixel_size()
pixel_size

results = {}

for channel_name in channel_names:
    recommendations = bg_subtractor.get_recommended_parameters(channel_name)
    print(f"Recommendations for {channel_name}:")
    print(f"  Method: {recommendations['method']}")
    print(f"  Radius: {recommendations['radius']}")
    print(f"  Description: {recommendations['description']}")
    print()

Recommendations for LAMP1:
  Method: rolling_ball
  Radius: 30
  Description: LAMP1 punctate structures - medium radius for vesicle preservation

Recommendations for Phalloidin:
  Method: rolling_ball
  Radius: 20
  Description: Phalloidin/actin - smaller radius to preserve filament structures

Recommendations for ALIX:
  Method: rolling_ball
  Radius: 30
  Description: Protein marker - standard radius for punctate/diffuse patterns

Recommendations for DAPI:
  Method: rolling_ball
  Radius: 50
  Description: DAPI/nuclear staining - rolling ball with large radius for nuclear regions



In [ ]:
# Prepare images dictionary for batch processing
images_dict = {}
for i, channel_name in enumerate(channel_names):
    images_dict[channel_name] = loaded_data[:, :, :, i]

# Process all channels at once
batch_results = bg_subtractor.batch_process(
    images=images_dict,
    pixel_size=pixel_size
)

# Access results
for channel_name, (corrected_img, metadata) in batch_results.items():
    print(f"{channel_name}: {corrected_img.shape}, method={metadata['method']}")


INFO:perinuclear_analysis.imaging_preprocessing.background_subtraction.background_subtractor:Processing channel: LAMP1
INFO:perinuclear_analysis.imaging_preprocessing.background_subtraction.background_subtractor:Applying rolling_ball background subtraction to 3D image (23, 1800, 1800)
INFO:perinuclear_analysis.imaging_preprocessing.background_subtraction.background_subtractor:Calculated chunk size: 23 slices (18.5 MB per slice)
